# 如何使用输出修正解析器
该[输出解析器](/docs/concepts/output_parsers/)会封装另一个输出解析器，当第一个解析器失败时，它会调用另一个LLM来修复任何错误。
但我们除了抛出错误之外，还可以做其他事情。具体来说，我们可以将格式错误的输出连同格式化指令一起传递给模型，并要求它进行修正。
在本示例中，我们将使用上述Pydantic输出解析器。以下是当我们传入一个不符合模式的结果时发生的情况：

In [1]:
from typing import List

from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [2]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [3]:
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [4]:
try:
    parser.parse(misformatted)
except OutputParserException as e:
    print(e)

Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE


现在我们可以构建并使用一个`OutputFixingParser`（输出修正解析器）。该输出解析器需要接收另一个输出解析器作为参数，同时还需要一个用于尝试修正任何格式错误的LLM（大语言模型）。

In [5]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [6]:
new_parser.parse(misformatted)

Actor(name='Tom Hanks', film_names=['Forrest Gump'])

查找 [OutputFixingParser](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.fix.OutputFixingParser.html#langchain.output_parsers.fix.OutputFixingParser) 的 API 文档。